In [1]:
import simpy
import numpy as np
import pandas as pd

In [2]:
#Parameter definieren 
ICU_BEDS = 47            #verfügbare Intensivbetten Für Corona Patienten in Nürnberg
NORM_BEDS = 400          #verfügbare Betten auf Normalstationen in Nürnberg
TOTAL_BEDS = ICU_BEDS + NORM_BEDS #Komplete Bettkapazität für Corona Patienten


MEAN_LOS = 14.0          # Mittelwert der Liegedauer in Tagen
BASE_SEED = 42
R = 20                   # Replikationen
SIM_DURATION = 150       # Simulationszeitraum in Tagen


In [ ]:
#Simulation eines Patienten vorerst wird eine Vorbelastung geschaffen und dann fangen die echten Patienten an

def patient_process(env, hospital, mean_los, rng, wait_times, is_preloaded=False):
    """
    Simuliert den diskreten Ereignisablauf eines Patienten (Entity) im System.

    Input-Kriterien & Parameter:
    ---------------------------
    env : simpy.Environment
        Die zentrale Instanz der Simulation, die die virtuelle Zeit (env.now) verwaltet.
    hospital : simpy.Resource
        Die begrenzte Ressource 'Bett'. Repräsentiert die Systemkapazität (TOTAL_BEDS).
        Ein Patient muss diese anfordern (request), bevor die Behandlung beginnt.
    mean_los : float
        Der Input für die mittlere Verweildauer (Mean Length of Stay). 
        Dient als Skalenparameter für die Exponentialverteilung.
    rng : numpy.random.Generator
        Der lokalisierte Zufallsgenerator. Verarbeitet den Seed der jeweiligen 
        Replikation, um die Varianz in der Aufenthaltsdauer zu erzeugen.
    wait_times : list
        Der Zielspeicher (Output-Container). Hier wird die Differenz zwischen 
        Ankunft und Aufnahme (t_aufnahme - t_ankunft) für die spätere 
        statistische Auswertung abgelegt.
    is_preloaded : bool, optional (Default: False)
        Steuerungs-Input zur Vermeidung von Bias. Patienten, die bereits bei t=0
        im System sind (Vorbelastung), haben per Definition nicht gewartet.
        Falls True, erfolgt kein Eintrag in 'wait_times'.

    Prozess-Logik:
    --------------
    1. Registrierung der Ankunftszeit (Timestamping).
    2. Warteschlange: Der Prozess pausiert (yield), bis ein Bett frei wird.
    3. Belegung: Berechnung der Wartezeit und Start der Behandlung.
    4. Service-Zeit: Generierung einer Zufallszahl (Exponentialverteilung):
       f(x; 1/mu) = (1/mu) * e^(-(1/mu)x).
    5. Freigabe: Das Bett wird nach Ablauf der Zeit automatisch wieder freigegeben.
    """
    t_ankunft = env.now
    with hospital.request() as req:
        yield req
        t_aufnahme = env.now
        
        # Statistik nur für reguläre Patienten (Ausschluss der Vorbelastung)
        if not is_preloaded:
            wait_times.append(t_aufnahme - t_ankunft)
        
        yield env.timeout(rng.exponential(mean_los))


In [ ]:
def setup_hospital(env, hospital, rng, wait_times, patients_per_week):
    """
    Steuert die Lastgenerierung basierend auf den Ankunftsraten.

    Input-Kriterien:
    - env: SimPy Environment (Zentraler Zeitgeber).
    - hospital: Resource-Objekt mit definierter Kapazität 'TOTAL_BEDS'.
    - rng: Ein numpy Random Generator. Nutzt den BASE_SEED, um 
           stochastische Varianz zwischen Replikationen zu ermöglichen.
    - wait_times: Referenz auf eine Liste (Mutable), die als Datenspeicher 
                  für die Zielgröße 'Wartezeit' fungiert.

    Logik der Input-Verarbeitung:
    Die Funktion transformiert die Rate (Patients/Week) in einen 
    Interarrival-Zeitraum (1/lambda) für einen Poisson-Prozess.
    """
    mean_interarrival = 7.0 / patients_per_week
    
    # Vorbelastung (Steady-State): 75% der Betten zum Start belegen
    preload_count = int(TOTAL_BEDS * 0.75)
    for _ in range(preload_count):
        env.process(patient_process(env, hospital, MEAN_LOS, rng, wait_times, is_preloaded=True))
    
    # Kontinuierliche Ankünfte
    while True:
        yield env.timeout(rng.exponential(mean_interarrival))
        env.process(patient_process(env, hospital, MEAN_LOS, rng, wait_times))


In [ ]:
def run_simulation(patients_per_week):
    """
    Kapselt ein einzelnes Szenario der Monte-Carlo-Simulation.

    Input-Kriterien:
    - num_beds: Unabhängige Variable (Entscheidungsvariable). Hier wird 
                getestet, ob 447 Betten für das Patientenaufkommen ausreichen.
    - seed: Deterministischer Input für stochastische Prozesse. Gewährleistet, 
            dass Experimente reproduzierbar sind (Variance Reduction Technique).

    Output-Kriterien:
    - Liefert Schätzwerte für Erwartungswert (Mean) und Risiko (95%-Quantil).
    """
    rep_means = []
    rep_q95s = []
    
    for i in range(R):
        # Saubere RNG-Nutzung 
        rng = np.random.default_rng(BASE_SEED + i)
        env = simpy.Environment()
        hospital = simpy.Resource(env, capacity=TOTAL_BEDS)
        wait_times = []
        
        env.process(setup_hospital(env, hospital, rng, wait_times, patients_per_week))
        env.run(until=SIM_DURATION)
        
        if wait_times:
            rep_means.append(np.mean(wait_times))
            rep_q95s.append(np.quantile(wait_times, 0.95))
            
    # Aggregation über die Replikate (Unsicherheitsquantifizierung)
    m_final = np.mean(rep_means)
    se_final = np.std(rep_means, ddof=1) / np.sqrt(R) # SE über Replikate
    q95_final = np.mean(rep_q95s)                    # Ø 95%-Quantil
    ci = (m_final - 1.96 * se_final, m_final + 1.96 * se_final)
    
    return m_final, se_final, ci, q95_final

# --- 2. Durchführung Szenarien-Vergleich ---
# 35/Woche = Normal, 235/Woche = Systemlimit 
szenarien = [35, 235]
results = []

for p_week in szenarien:
    m, se, ci, q95 = run_simulation(p_week)
    rho = ((p_week / 7.0) * MEAN_LOS) / TOTAL_BEDS # Theoretische Auslastung
    
    results.append({
        "Szenario": "Normal" if p_week == 35 else "Stress-Test",
        "Pat./Woche": p_week,
        "Auslastung (ρ)": f"{rho*100:.1f}%",
        "Mean Wartezeit": f"{m:.4f} d",
        "Std.-Fehler (SE)": f"{se:.4f}",
        "95%-Konf.-Intervall": f"[{ci[0]:.3f}, {ci[1]:.3f}]",
        "Ø 95%-Quantil": f"{q95:.4f} d"
    })



In [6]:
#Output

print("\nERGEBNISSE DER KRANKENHAUS-SIMULATION (Weg A: Lastvariation)")
print("=" * 95)
df_output = pd.DataFrame(results)
print(df_output.to_string(index=False))
print("-" * 95)
print(f"Bedingungen: Betten = {TOTAL_BEDS}, Replikationen R = {R}, Dauer = {SIM_DURATION} Tage.")


ERGEBNISSE DER KRANKENHAUS-SIMULATION (Weg A: Lastvariation)
   Szenario  Pat./Woche Auslastung (ρ) Mean Wartezeit Std.-Fehler (SE) 95%-Konf.-Intervall Ø 95%-Quantil
     Normal          35          15.7%       0.0000 d           0.0000      [0.000, 0.000]      0.0000 d
Stress-Test         235         105.1%       2.7425 d           0.2882      [2.178, 3.307]      6.4597 d
-----------------------------------------------------------------------------------------------
Bedingungen: Betten = 447, Replikationen R = 20, Dauer = 150 Tage.
